In [590]:
import csv
with open("LAB1.csv", 'r') as f:
  reader = csv.reader(f)
  comp_data = list(reader)
comp_data.remove(['age', 'income', 'student', 'credit rating', 'buys computer'])
comp_data

[['<=30', 'high', 'no', 'fair', 'no'],
 ['<=30', 'high ', 'no', 'excellent', 'no'],
 ['31...40', 'high', 'no', 'fair', 'yes'],
 ['>40', 'medium', 'no', 'fair', 'yes'],
 ['>40', 'low', 'yes', 'fair', 'yes'],
 ['>40', 'low', 'yes', 'excellent', 'no'],
 ['31...40', 'low', 'yes', 'excellent', 'yes'],
 ['<=30', 'medium', 'no', 'fair', 'no'],
 ['<=30', 'low', 'yes', 'fair', 'yes'],
 ['>40', 'medium', 'yes', 'fair', 'yes'],
 ['<=30', 'medium', 'yes', 'excellent', 'yes'],
 ['31...40', 'medium', 'no', 'excellent', 'yes'],
 ['31...40', 'high', 'yes', 'fair', 'yes'],
 ['>40', 'medium', 'no', 'excellent', 'no']]

In [591]:
columns = ['age', 'income', 'student', 'credit rating', 'buys computer']

In [592]:
def count_labels(rows):
    count = {}  # a dictionary of label -> count.
    for row in rows:
        # target is in last column
        label = row[-1]
        if label not in count:
            count[label] = 0
        count[label] += 1
    return count

In [593]:
def is_num(value):
    return isinstance(value, int) or isinstance(value, float)

In [594]:
class Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        val = example[self.column]
        if is_num(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        condition = "=="
        if is_num(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            columns[self.column], condition, str(self.value))

In [595]:
Question(1, 'high')

Is income == high?

In [596]:
test_rec = comp_data[0]
print(test_rec)
q = Question(1, 'high')
q.match(test_rec) 

['<=30', 'high', 'no', 'fair', 'no']


True

In [597]:
def partition(rows, question):
    #For every record in the dataset, check if it answers question. If yes add to true list, else add to false list.
    t_rows, f_rows = [], []
    for row in rows:
        if question.match(row):
            t_rows.append(row)
        else:
            f_rows.append(row)
    return t_rows, f_rows

In [598]:
#######
# Demo:
# Let's partition the training data based on whether rows are Red.
true_rows, false_rows = partition(comp_data, Question(2, 'yes'))
# This will contain all the 'Red' rows.
true_rows

[['>40', 'low', 'yes', 'fair', 'yes'],
 ['>40', 'low', 'yes', 'excellent', 'no'],
 ['31...40', 'low', 'yes', 'excellent', 'yes'],
 ['<=30', 'low', 'yes', 'fair', 'yes'],
 ['>40', 'medium', 'yes', 'fair', 'yes'],
 ['<=30', 'medium', 'yes', 'excellent', 'yes'],
 ['31...40', 'high', 'yes', 'fair', 'yes']]

In [599]:
false_rows

[['<=30', 'high', 'no', 'fair', 'no'],
 ['<=30', 'high ', 'no', 'excellent', 'no'],
 ['31...40', 'high', 'no', 'fair', 'yes'],
 ['>40', 'medium', 'no', 'fair', 'yes'],
 ['<=30', 'medium', 'no', 'fair', 'no'],
 ['31...40', 'medium', 'no', 'excellent', 'yes'],
 ['>40', 'medium', 'no', 'excellent', 'no']]

In [600]:
def gini(recs):
    counts = count_labels(recs)
    impurity = 1
    for i in counts:
        prob_of_i = counts[i] / float(len(recs))
        impurity = impurity - (prob_of_i)**2
    return impurity

In [601]:
def info_gain(l, r, current_impurity):
    #The impurity of the root/parent - weighted impurity of two child nodes.
    p = float(len(l)) / (len(l) + len(r))
    return current_impurity - p * gini(l) - (1 - p) * gini(r)

In [602]:
current_imp = gini(comp_data)
current_imp

0.4591836734693877

In [603]:
true_rows, false_rows = partition(comp_data, Question(3, 'fair'))
info_gain(true_rows, false_rows, current_uncertainty)

0.030612244897959134

In [604]:
def find_best_div(rows):
    #Iterating over every feature and calculating the information gain to find best question.
    max_gain = 0 
    best_question = None  
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  

    for col in range(n_features):  
        values = set([row[col] for row in rows])  

        for val in values:  # for each value

            question = Question(col, val)

            true_rows, false_rows = partition(rows, question)

            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # IG from this division by question
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            if gain >= max_gain:
                max_gain, best_question = gain, question

    return max_gain, best_question

In [605]:
max_gain, best_question = find_best_div(comp_data)
best_question

Is age == 31...40?

In [606]:
class Leaf:
    #Dictionary of label -> frequency in data set to reach this leaf.
    def __init__(self, rows):
        self.predictions = count_labels(rows)

In [607]:
class Decision_Node:
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [608]:
def build_tree(rows):

    #partition, calculate IG and return best question to start with.
    gain, question = find_best_div(rows)

    # Base case: no further info gain, hence leaf.
    if gain == 0:
        return Leaf(rows)

    # Found question, partition based on it.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    return Decision_Node(question, true_branch, false_branch)

In [609]:
def print_tree(node, spacing=""):

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [610]:
dec_tree = build_tree(comp_data)
print_tree(dec_tree)

Is age == 31...40?
--> True:
  Predict {'yes': 4}
--> False:
  Is student == no?
  --> True:
    Is age == >40?
    --> True:
      Is credit rating == excellent?
      --> True:
        Predict {'no': 1}
      --> False:
        Predict {'yes': 1}
    --> False:
      Predict {'no': 3}
  --> False:
    Is credit rating == excellent?
    --> True:
      Is income == low?
      --> True:
        Predict {'no': 1}
      --> False:
        Predict {'yes': 1}
    --> False:
      Predict {'yes': 3}


In [611]:
def classify(rec, node):

    # Base case: leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Follow the true-branch or the false-branch.
    if node.question.match(rec):
        return classify(rec, node.true_branch)
    else:
        return classify(rec, node.false_branch)

In [612]:
comp_data

[['<=30', 'high', 'no', 'fair', 'no'],
 ['<=30', 'high ', 'no', 'excellent', 'no'],
 ['31...40', 'high', 'no', 'fair', 'yes'],
 ['>40', 'medium', 'no', 'fair', 'yes'],
 ['>40', 'low', 'yes', 'fair', 'yes'],
 ['>40', 'low', 'yes', 'excellent', 'no'],
 ['31...40', 'low', 'yes', 'excellent', 'yes'],
 ['<=30', 'medium', 'no', 'fair', 'no'],
 ['<=30', 'low', 'yes', 'fair', 'yes'],
 ['>40', 'medium', 'yes', 'fair', 'yes'],
 ['<=30', 'medium', 'yes', 'excellent', 'yes'],
 ['31...40', 'medium', 'no', 'excellent', 'yes'],
 ['31...40', 'high', 'yes', 'fair', 'yes'],
 ['>40', 'medium', 'no', 'excellent', 'no']]

In [613]:
d = classify(comp_data[7], dec_tree)
list(d.keys())

['no']

In [614]:
def p_leaf(c):
    t = sum(c.values()) * 1.0
    prob = {}
    for l in c.keys():
        prob[l] = str(int(c[l] / t * 100)) + "%"
    return prob

In [615]:
test_data = [['<=30', 'high', 'no', 'fair', 'no'],['31...40', 'medium', 'no', 'fair', 'yes'],['>40', 'medium', 'yes', 'fair', 'no'],['<=30', 'low', 'yes', 'excellent', 'no'],['31...40', 'medium', 'yes', 'excellent', 'yes']]
for rec in test_data:
    print ("Real: %s. Predicted: %s" %
           (rec[-1], print_leaf(classify(rec, dec_tree))))

Real: no. Predicted: {'no': '100%'}
Real: yes. Predicted: {'yes': '100%'}
Real: no. Predicted: {'yes': '100%'}
Real: no. Predicted: {'no': '100%'}
Real: yes. Predicted: {'yes': '100%'}


In [616]:
total = len(test_data)
positive = 0
for rec in test_data:
    d = classify(rec, dec_tree)
    l = list(d.keys())
    if(rec[-1]==l[0]):
        positive += 1
accuracy = positive/total
print('Accuracy =',accuracy)

Accuracy = 0.8
